## 기본 세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Wed Nov 30 04:16:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
os.chdir("/content/drive/MyDrive")

In [4]:
!unzip "/content/drive/MyDrive/petdata_disjoint.zip" -d "/content/drive/MyDrive"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_82.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_83.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_84.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_85.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_86.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_87.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_88.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_89.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_9.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_90.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_91.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_92.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_terrier_93.jpg  
  inflating: /content/drive/MyDrive/test/31/scottish_te

In [5]:
!unzip "/content/drive/MyDrive/petdata_nodisjoint.zip" -d "/content/drive/MyDrive"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/test/30/samoyed_58.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_59.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_6.jpg  
 extracting: /content/drive/MyDrive/test/30/samoyed_60.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_61.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_62.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_63.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_64.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_65.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_66.jpg  
 extracting: /content/drive/MyDrive/test/30/samoyed_67.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_68.jpg  
 extracting: /content/drive/MyDrive/test/30/samoyed_69.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_7.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed_70.jpg  
  inflating: /content/drive/MyDrive/test/30/samoyed

## 모델 기본 세팅

In [7]:
from deeplab2.model.pixel_encoder import moat
import tensorflow as tf
import numpy as np

ModuleNotFoundError: ignored

In [ ]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.layers import Dense, Dropout, Conv2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

print(tf.test.gpu_device_name())

/device:GPU:0


In [ ]:
# releases the global state: avoid clutter from old models and layers
K.clear_session()

In [ ]:
# 초기 변수 설정
n_classes = 37

### 이미지 크기 설정
img_width, img_height = 256, 256

train_data_dir = '/content/drive/MyDrive/train'
train1_data_dir = '/content/drive/MyDrive/train1'
train2_data_dir = '/content/drive/MyDrive/train2'
validation_data_dir = '/content/drive/MyDrive/validation'
validation_data2_dir = '/content/drive/MyDrive/validation'
test_data_dir = '/content/drive/MyDrive/test'

nb_train_samples =  1656*2
nb_train1_samples =  1656
nb_train2_samples =  1656   
nb_validation_samples = 368
nb_validation2_samples = 368    
nb_test_samples = 3668  
batch_size = 16

In [ ]:
# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)


train_datagen = ImageDataGenerator(
    rescale=1./ 255,       # multiply the data by the value
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')

train1_datagen = ImageDataGenerator(
    rescale=1./ 255,       # multiply the data by the value
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')   

# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)
train2_datagen = ImageDataGenerator(
    rescale=1./ 255,       # multiply the data by the value
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')   

val_datagen2 = ImageDataGenerator(rescale=1./ 255)
val_datagen = ImageDataGenerator(rescale=1./ 255)
test_datagen = ImageDataGenerator(rescale=1./ 255)

In [ ]:
# flow_from_directory : Numpy Array Iterator 객체 생성
# 인자로 설정해주는 directory의 바로 하위 디렉토리 이름을 레이블이라고 간주, 그 디렉토리 아래의 파일들을 해당 레이블의 이미지들이라고 알아서 추측


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


train1_generator = train1_datagen.flow_from_directory(
    train1_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

train2_generator = train2_datagen.flow_from_directory(
    train2_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')
    
  
validation2_generator = val_datagen.flow_from_directory(
    validation_data2_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode=None)

Found 3312 images belonging to 37 classes.
Found 1656 images belonging to 37 classes.
Found 1656 images belonging to 37 classes.
Found 368 images belonging to 37 classes.
Found 368 images belonging to 37 classes.
Found 3668 images belonging to 37 classes.


In [6]:
!pip install adabelief_tf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 모델1 Disjoint two model

In [ ]:
moat1=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

In [ ]:
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

In [ ]:
from tensorflow.keras.layers import (Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten)
from tensorflow import keras

from adabelief_tf import AdaBeliefOptimizer
opt = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=1e-14, rectify=True)

# tf.keras.optimizers.schedules.LearningRateSchedule 나중에. 오목한 형태로 하는게 좋을듯 함.

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



In [ ]:
y11 = moat1.output["stage1"]
y12 = moat1.output["res1"]
y21 = moat1.output["stage2"]
y22 = moat1.output["res2"]
y31 = moat1.output["stage3"]
y32 = moat1.output["res3"]
y41 = moat1.output["stage4"]
y42 = moat1.output["res4"]
y51 = moat1.output["stage5"]
y52 = moat1.output["res5"]

z1 = y11+y12
z2 = y21+y22
z3 = y31+y32
z4 = y41+y42
z5 = y51+y52

In [ ]:
a1=Conv2D(8,(1,1),activation='relu')(z1)
a2=Conv2D(8,(3,3),activation='relu')(a1)
a3=MaxPooling2D(2,2)(a2) 
a4=Conv2D(8,(3,3),activation='relu')(a3)
a5=MaxPooling2D(2,2)(a4) 
a6=Conv2D(8,(3,3),activation='relu')(a5)
a7=MaxPooling2D(2,2)(a6) 
a8=Conv2D(8,(3,3),activation='relu')(a7)
a9=MaxPooling2D(2,2)(a8)
a10=Conv2D(8,(3,3),activation='relu')(a9)
a11=MaxPooling2D(2,2)(a10)
a12=Conv2D(32,(1,1),activation='relu')(a11)
aa1=Flatten(name='flatten_layer1')(a12)

b1=Conv2D(8,(1,1),activation='relu')(z2)
b2=Conv2D(8,(3,3),activation='relu')(b1)
b3=MaxPooling2D(2,2)(b2) 
b4=Conv2D(8,(3,3),activation='relu')(b3)
b5=MaxPooling2D(2,2)(b4) 
b6=Conv2D(8,(3,3),activation='relu')(b5)
b7=MaxPooling2D(2,2)(b6) 
b8=Conv2D(8,(3,3),activation='relu')(b7)
b9=MaxPooling2D(2,2)(b8)
b10=Conv2D(32,(1,1),activation='relu')(b9)
bb1=Flatten(name='flatten_layer2')(b10)

c1=Conv2D(16,(1,1),activation='relu')(z3)
c2=Conv2D(16,(3,3),activation='relu')(c1)
c3=MaxPooling2D(2,2)(c2) 
c4=Conv2D(16,(3,3),activation='relu')(c3)
c5=MaxPooling2D(2,2)(c4) 
c6=Conv2D(16,(3,3),activation='relu')(c5)
c7=MaxPooling2D(2,2)(c6) 
c8=Conv2D(64,(1,1),activation='relu')(c7)
cc1=Flatten(name='flatten_layer3')(c8)

d1=Conv2D(32,(1,1),activation='relu')(z4)
d2=Conv2D(32,(3,3),activation='relu')(d1)
d3=MaxPooling2D(2,2)(d2) 
d4=Conv2D(32,(3,3),activation='relu')(d3)
d5=MaxPooling2D(2,2)(d4) 
d6=Conv2D(128,(1,1),activation='relu')(d5)
dd1=Flatten(name='flatten_layer4')(d6)

e1=Conv2D(64,(1,1),activation='relu')(z5)
e2=Conv2D(64,(3,3),activation='relu')(e1)
e3=MaxPooling2D(2,2)(e2) 
e4=Conv2D(64,(2,2),activation='relu')(e3)
e5=Conv2D(256,(1,1),activation='relu')(e4)
ee1=Flatten(name='flatten_layer5')(e5)

fc1=tf.concat([aa1,bb1,cc1,dd1,ee1],axis=1)
out1=Dense(units=37, activation='softmax')(fc1)



def create_model():
  return tf.keras.models.Model(moat1.input, out1)

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")


with tf.device('/device:GPU:0'):
  model1 = create_model()
 # model1.load_weights('model_fulltrain5.hdf5')
 # model1.layers[1]=moat2

  model1.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='model1_disjoint_train1.hdf5', verbose=1, save_best_only=False)
  csv_logger = CSVLogger('model1_disjoint_train1.log')
  history = model1.fit_generator(train1_generator, 
                    steps_per_epoch = nb_train1_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=200,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
  
  model1.save('model1_disjoint_train1.h5')


In [ ]:
moat2=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

In [ ]:
k11 = moat2.output["stage1"]
k12 = moat2.output["res1"]
k21 = moat2.output["stage2"]
k22 = moat2.output["res2"]
k31 = moat2.output["stage3"]
k32 = moat2.output["res3"]
k41 = moat2.output["stage4"]
k42 = moat2.output["res4"]
k51 = moat2.output["stage5"]
k52 = moat2.output["res5"]

l1 = k11+k12
l2 = k21+k22
l3 = k31+k32
l4 = k41+k42
l5 = k51+k52

n1=Conv2D(8,(1,1),activation='relu')(l1)
n2=Conv2D(8,(3,3),activation='relu')(n1)
n3=MaxPooling2D(2,2)(n2) 
n4=Conv2D(8,(3,3),activation='relu')(n3)
n5=MaxPooling2D(2,2)(n4) 
n6=Conv2D(8,(3,3),activation='relu')(n5)
n7=MaxPooling2D(2,2)(n6) 
n8=Conv2D(8,(3,3),activation='relu')(n7)
n9=MaxPooling2D(2,2)(n8)
n10=Conv2D(8,(3,3),activation='relu')(n9)
n11=MaxPooling2D(2,2)(n10)
n12=Conv2D(32,(1,1),activation='relu')(n11)
nn1=Flatten(name='flatten_layer1')(n12)

m1=Conv2D(8,(1,1),activation='relu')(l2)
m2=Conv2D(8,(3,3),activation='relu')(m1)
m3=MaxPooling2D(2,2)(m2) 
m4=Conv2D(8,(3,3),activation='relu')(m3)
m5=MaxPooling2D(2,2)(m4) 
m6=Conv2D(8,(3,3),activation='relu')(m5)
m7=MaxPooling2D(2,2)(m6) 
m8=Conv2D(8,(3,3),activation='relu')(m7)
m9=MaxPooling2D(2,2)(m8)
m10=Conv2D(32,(1,1),activation='relu')(m9)
mm1=Flatten(name='flatten_layer2')(m10)

o1=Conv2D(16,(1,1),activation='relu')(l3)
o2=Conv2D(16,(3,3),activation='relu')(o1)
o3=MaxPooling2D(2,2)(o2) 
o4=Conv2D(16,(3,3),activation='relu')(o3)
o5=MaxPooling2D(2,2)(o4) 
o6=Conv2D(16,(3,3),activation='relu')(o5)
o7=MaxPooling2D(2,2)(o6) 
o8=Conv2D(64,(1,1),activation='relu')(o7)
oo1=Flatten(name='flatten_layer3')(o8)

p1=Conv2D(32,(1,1),activation='relu')(l4)
p2=Conv2D(32,(3,3),activation='relu')(p1)
p3=MaxPooling2D(2,2)(p2) 
p4=Conv2D(32,(3,3),activation='relu')(p3)
p5=MaxPooling2D(2,2)(p4) 
p6=Conv2D(128,(1,1),activation='relu')(p5)
pp1=Flatten(name='flatten_layer4')(p6)

q1=Conv2D(64,(1,1),activation='relu')(l5)
q2=Conv2D(64,(3,3),activation='relu')(q1)
q3=MaxPooling2D(2,2)(q2) 
q4=Conv2D(64,(2,2),activation='relu')(q3)
q5=Conv2D(256,(1,1),activation='relu')(q4)
qq1=Flatten(name='flatten_layer5')(q5)

fc2=tf.concat([nn1,mm1,oo1,pp1,qq1],axis=1)
out2=Dense(units=37, activation='softmax')(fc2)

def create_model2():
  return tf.keras.models.Model(moat2.input, out2)

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")


with tf.device('/device:GPU:0'):
  model2 = create_model2()
 # model1.load_weights('model_fulltrain5.hdf5')
 # model1.layers[1]=moat2

  model2.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer2 = ModelCheckpoint(filepath='model2_disjoint_train2.hdf5', verbose=1, save_best_only=False)
  csv_logger2 = CSVLogger('model2_disjoint_train2.log')
  history2 = model2.fit_generator(train2_generator, 
                    steps_per_epoch = nb_train2_samples // batch_size,      
                    validation_data=validation2_generator,
                    validation_steps=nb_validation2_samples // batch_size,  
                    epochs=200,                                             
                    verbose=1,
                    callbacks=[csv_logger2, checkpointer2])
  
  model2.save('model2_disjoint_train2.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/200
51/51 [==============================] - ETA: 0s - loss: 4.3205 - sparse_categorical_accuracy: 0.0283
Epoch 1: saving model to model2_disjoint_train2.hdf5
51/51 [==============================] - 124s 917ms/step - loss: 4.3205 - sparse_categorical_accuracy: 0.0283 - val_loss: 4.2627 - val_sparse_categorical_accuracy: 0.0227
Epoch 2/200
51/51 [==============================] - ETA: 0s - loss: 4.1860 - sparse_categorical_accuracy: 0.0209
Epoch 2: saving model to model2_disjoint_train2.hdf5
51/51 [==============================] - 44s 845ms/step - loss: 4.1860 - sparse_categorical_accuracy: 0.0209 - val_loss: 3.9935 - val_sparse_categorical_accuracy: 0.0284
Epoch 3/200
51/51 [==============================] - ETA: 0s - loss: 4.0453 - sparse_categorical_accuracy: 0.0333
Epoch 3: saving model to model2_disjoint_train2.hdf5
51/51 [==============================] - 43s 842ms/step - loss: 4.0453 - sparse_categorical_accuracy: 0.0333 - val_loss: 3.9519 - val_sparse_categorical_accur

### model1 앙상블

In [ ]:
moat1=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

moat2=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

y11 = moat1.output["stage1"]
y12 = moat1.output["res1"]
y21 = moat1.output["stage2"]
y22 = moat1.output["res2"]
y31 = moat1.output["stage3"]
y32 = moat1.output["res3"]
y41 = moat1.output["stage4"]
y42 = moat1.output["res4"]
y51 = moat1.output["stage5"]
y52 = moat1.output["res5"]

z1 = y11+y12
z2 = y21+y22
z3 = y31+y32
z4 = y41+y42
z5 = y51+y52

a1=Conv2D(8,(1,1),activation='relu')(z1)
a2=Conv2D(8,(3,3),activation='relu')(a1)
a3=MaxPooling2D(2,2)(a2) 
a4=Conv2D(8,(3,3),activation='relu')(a3)
a5=MaxPooling2D(2,2)(a4) 
a6=Conv2D(8,(3,3),activation='relu')(a5)
a7=MaxPooling2D(2,2)(a6) 
a8=Conv2D(8,(3,3),activation='relu')(a7)
a9=MaxPooling2D(2,2)(a8)
a10=Conv2D(8,(3,3),activation='relu')(a9)
a11=MaxPooling2D(2,2)(a10)
a12=Conv2D(32,(1,1),activation='relu')(a11)
aa1=Flatten(name='flatten_layer1')(a12)

b1=Conv2D(8,(1,1),activation='relu')(z2)
b2=Conv2D(8,(3,3),activation='relu')(b1)
b3=MaxPooling2D(2,2)(b2) 
b4=Conv2D(8,(3,3),activation='relu')(b3)
b5=MaxPooling2D(2,2)(b4) 
b6=Conv2D(8,(3,3),activation='relu')(b5)
b7=MaxPooling2D(2,2)(b6) 
b8=Conv2D(8,(3,3),activation='relu')(b7)
b9=MaxPooling2D(2,2)(b8)
b10=Conv2D(32,(1,1),activation='relu')(b9)
bb1=Flatten(name='flatten_layer2')(b10)

c1=Conv2D(16,(1,1),activation='relu')(z3)
c2=Conv2D(16,(3,3),activation='relu')(c1)
c3=MaxPooling2D(2,2)(c2) 
c4=Conv2D(16,(3,3),activation='relu')(c3)
c5=MaxPooling2D(2,2)(c4) 
c6=Conv2D(16,(3,3),activation='relu')(c5)
c7=MaxPooling2D(2,2)(c6) 
c8=Conv2D(64,(1,1),activation='relu')(c7)
cc1=Flatten(name='flatten_layer3')(c8)

d1=Conv2D(32,(1,1),activation='relu')(z4)
d2=Conv2D(32,(3,3),activation='relu')(d1)
d3=MaxPooling2D(2,2)(d2) 
d4=Conv2D(32,(3,3),activation='relu')(d3)
d5=MaxPooling2D(2,2)(d4) 
d6=Conv2D(128,(1,1),activation='relu')(d5)
dd1=Flatten(name='flatten_layer4')(d6)

e1=Conv2D(64,(1,1),activation='relu')(z5)
e2=Conv2D(64,(3,3),activation='relu')(e1)
e3=MaxPooling2D(2,2)(e2) 
e4=Conv2D(64,(2,2),activation='relu')(e3)
e5=Conv2D(256,(1,1),activation='relu')(e4)
ee1=Flatten(name='flatten_layer5')(e5)

fc1=tf.concat([aa1,bb1,cc1,dd1,ee1],axis=1)
out1=Dense(units=37, activation='softmax')(fc1)

k11 = moat2.output["stage1"]
k12 = moat2.output["res1"]
k21 = moat2.output["stage2"]
k22 = moat2.output["res2"]
k31 = moat2.output["stage3"]
k32 = moat2.output["res3"]
k41 = moat2.output["stage4"]
k42 = moat2.output["res4"]
k51 = moat2.output["stage5"]
k52 = moat2.output["res5"]

l1 = k11+k12
l2 = k21+k22
l3 = k31+k32
l4 = k41+k42
l5 = k51+k52

n1=Conv2D(8,(1,1),activation='relu')(l1)
n2=Conv2D(8,(3,3),activation='relu')(n1)
n3=MaxPooling2D(2,2)(n2) 
n4=Conv2D(8,(3,3),activation='relu')(n3)
n5=MaxPooling2D(2,2)(n4) 
n6=Conv2D(8,(3,3),activation='relu')(n5)
n7=MaxPooling2D(2,2)(n6) 
n8=Conv2D(8,(3,3),activation='relu')(n7)
n9=MaxPooling2D(2,2)(n8)
n10=Conv2D(8,(3,3),activation='relu')(n9)
n11=MaxPooling2D(2,2)(n10)
n12=Conv2D(32,(1,1),activation='relu')(n11)
nn1=Flatten(name='flatten_layer1')(n12)

m1=Conv2D(8,(1,1),activation='relu')(l2)
m2=Conv2D(8,(3,3),activation='relu')(m1)
m3=MaxPooling2D(2,2)(m2) 
m4=Conv2D(8,(3,3),activation='relu')(m3)
m5=MaxPooling2D(2,2)(m4) 
m6=Conv2D(8,(3,3),activation='relu')(m5)
m7=MaxPooling2D(2,2)(m6) 
m8=Conv2D(8,(3,3),activation='relu')(m7)
m9=MaxPooling2D(2,2)(m8)
m10=Conv2D(32,(1,1),activation='relu')(m9)
mm1=Flatten(name='flatten_layer2')(m10)

o1=Conv2D(16,(1,1),activation='relu')(l3)
o2=Conv2D(16,(3,3),activation='relu')(o1)
o3=MaxPooling2D(2,2)(o2) 
o4=Conv2D(16,(3,3),activation='relu')(o3)
o5=MaxPooling2D(2,2)(o4) 
o6=Conv2D(16,(3,3),activation='relu')(o5)
o7=MaxPooling2D(2,2)(o6) 
o8=Conv2D(64,(1,1),activation='relu')(o7)
oo1=Flatten(name='flatten_layer3')(o8)

p1=Conv2D(32,(1,1),activation='relu')(l4)
p2=Conv2D(32,(3,3),activation='relu')(p1)
p3=MaxPooling2D(2,2)(p2) 
p4=Conv2D(32,(3,3),activation='relu')(p3)
p5=MaxPooling2D(2,2)(p4) 
p6=Conv2D(128,(1,1),activation='relu')(p5)
pp1=Flatten(name='flatten_layer4')(p6)

q1=Conv2D(64,(1,1),activation='relu')(l5)
q2=Conv2D(64,(3,3),activation='relu')(q1)
q3=MaxPooling2D(2,2)(q2) 
q4=Conv2D(64,(2,2),activation='relu')(q3)
q5=Conv2D(256,(1,1),activation='relu')(q4)
qq1=Flatten(name='flatten_layer5')(q5)

fc2=tf.concat([nn1,mm1,oo1,pp1,qq1],axis=1)
out2=Dense(units=37, activation='softmax')(fc2)

model1=tf.keras.models.Model(moat1.input, out1)
model1.load_weights('model1_disjoint_train1.hdf5')

model2=tf.keras.models.Model(moat2.input, out2)
model2.load_weights('model2_disjoint_train2.hdf5')

In [ ]:
model1.trainable=False
model2.trainable=False

In [ ]:
model3=tf.keras.Model(model1.input, model1.layers[53].output)
model4=tf.keras.Model(model2.input, model2.layers[53].output)

In [ ]:
en_input = Input(shape=(256,256,3))
x = model3(en_input)
model5 = tf.keras.Model(en_input, x)

y = model4(en_input)
model6 = tf.keras.Model(en_input, y)

z= tf.concat([model5.output, model6.output],axis=1)
l= Dense(units=4096, activation='relu')(z)
l= Dense(units=2048, activation='relu')(l)
en_outs = Dense(units=37, activation='softmax')(l)

In [ ]:
## 한번만 실행
#for layer in model2.layers:
#    layer._name = layer.name + str("_2")

In [ ]:
opt = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=1e-14, rectify=True)

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



In [ ]:
def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_outs) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

with tf.device('/device:GPU:0'):
  en_model = create_model3()
  en_model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='ensemble_type1_1.tf', verbose=1, save_best_only=False)
  csv_logger = CSVLogger('ensemble_type1_1.log')
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
  
en_model.save('ensemble_type1_1.tf', save_format=tf)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
207/207 [==============================] - ETA: 0s - loss: 1.8852 - sparse_categorical_accuracy: 0.7594
Epoch 1: saving model to ensemble_type1_1.tf


207/207 [==============================] - 137s 569ms/step - loss: 1.8852 - sparse_categorical_accuracy: 0.7594 - val_loss: 2.7345 - val_sparse_categorical_accuracy: 0.7500
Epoch 2/30
207/207 [==============================] - ETA: 0s - loss: 1.3800 - sparse_categorical_accuracy: 0.8584
Epoch 2: saving model to ensemble_type1_1.tf


207/207 [==============================] - 115s 556ms/step - loss: 1.3800 - sparse_categorical_accuracy: 0.8584 - val_loss: 1.8372 - val_sparse_categorical_accuracy: 0.8342
Epoch 3/30
207/207 [==============================] - ETA: 0s - loss: 1.1389 - sparse_categorical_accuracy: 0.8647
Epoch 3: saving model to ensemble_type1_1.tf


207/207 [==============================] - 118s 569ms/step - loss: 1.1389 - sparse_categorical_accuracy: 0.8647 - val_loss: 1.6088 - val_sparse_categorical_accuracy: 0.7745
Epoch 4/30
207/207 [==============================] - ETA: 0s - loss: 0.5025 - sparse_categorical_accuracy: 0.9010
Epoch 4: saving model to ensemble_type1_1.tf


207/207 [==============================] - 112s 539ms/step - loss: 0.5025 - sparse_categorical_accuracy: 0.9010 - val_loss: 1.0737 - val_sparse_categorical_accuracy: 0.8071
Epoch 5/30
207/207 [==============================] - ETA: 0s - loss: 0.4080 - sparse_categorical_accuracy: 0.9139
Epoch 5: saving model to ensemble_type1_1.tf


207/207 [==============================] - 120s 578ms/step - loss: 0.4080 - sparse_categorical_accuracy: 0.9139 - val_loss: 0.8770 - val_sparse_categorical_accuracy: 0.8179
Epoch 6/30
207/207 [==============================] - ETA: 0s - loss: 0.3812 - sparse_categorical_accuracy: 0.9025
Epoch 6: saving model to ensemble_type1_1.tf


207/207 [==============================] - 110s 532ms/step - loss: 0.3812 - sparse_categorical_accuracy: 0.9025 - val_loss: 0.9811 - val_sparse_categorical_accuracy: 0.8043
Epoch 7/30
 62/207 [=======>......................] - ETA: 42s - loss: 0.4297 - sparse_categorical_accuracy: 0.8952

KeyboardInterrupt: ignored

## Model2 Two Disjoint Model

In [ ]:
opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=1e-14, rectify=True)

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



In [ ]:
moat2=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

k11 = moat2.output["stage1"]
k12 = moat2.output["res1"]
k21 = moat2.output["stage2"]
k22 = moat2.output["res2"]
k31 = moat2.output["stage3"]
k32 = moat2.output["res3"]
k41 = moat2.output["stage4"]
k42 = moat2.output["res4"]
k51 = moat2.output["stage5"]
k52 = moat2.output["res5"]

l1 = k11+k12
l2 = k21+k22
l3 = k31+k32
l4 = k41+k42
l5 = k51+k52

f1=Conv2D(32,(3,3),activation='relu')(l1)
f2=MaxPooling2D(2,2)(f1) 
f3=Conv2D(40,(3,3),activation='relu')(f2)
f4=MaxPooling2D(2,2)(f3) 
f5=Conv2D(48,(3,3),activation='relu')(f4)
f6=MaxPooling2D(2,2)(f5) 
f7=Conv2D(56,(3,3),activation='relu')(f6)
f8=MaxPooling2D(2,2)(f7)
f9=Conv2D(64,(3,3),activation='relu')(f8)
f10=MaxPooling2D(2,2)(f9)
f11=Conv2D(72,(2,2),activation='relu')(f10)
ff1=Flatten(name='flatten_layer1')(f11)

g1=Conv2D(32,(3,3),activation='relu')(l2)
g2=MaxPooling2D(2,2)(g1) 
g3=Conv2D(40,(3,3),activation='relu')(g2)
g4=MaxPooling2D(2,2)(g3) 
g5=Conv2D(48,(3,3),activation='relu')(g4)
g6=MaxPooling2D(2,2)(g5) 
g7=Conv2D(56,(3,3),activation='relu')(g6)
g8=MaxPooling2D(2,2)(g7)
g9=Conv2D(64,(2,2),activation='relu')(g8)
gg1=Flatten(name='flatten_layer2')(g9)

h1=Conv2D(64,(3,3),activation='relu')(l3)
h2=MaxPooling2D(2,2)(h1) 
h3=Conv2D(80,(3,3),activation='relu')(h2)
h4=MaxPooling2D(2,2)(h3) 
h5=Conv2D(96,(3,3),activation='relu')(h4)
h6=MaxPooling2D(2,2)(h5) 
h7=Conv2D(112,(2,2),activation='relu')(h6)
hh1=Flatten(name='flatten_layer3')(h7)

i1=Conv2D(128,(3,3),activation='relu')(l4)
i2=MaxPooling2D(2,2)(i1) 
i3=Conv2D(160,(3,3),activation='relu')(i2)
i4=MaxPooling2D(2,2)(i3) 
i5=Conv2D(200,(2,2),activation='relu')(i4)
ii1=Flatten(name='flatten_layer4')(i5)

j1=Conv2D(256,(3,3),activation='relu')(l5)
j2=MaxPooling2D(2,2)(j1) 
j3=Conv2D(320,(3,3),activation='relu')(j2)
jj1=Flatten(name='flatten_layer5')(j3)

fc2=tf.concat([ff1,gg1,hh1,ii1,jj1],axis=1)
out2=Dense(units=101, activation='softmax')(fc2)

def create_model():
  return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
  modelv2_1 = create_model()
  modelv2_1.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer1 = ModelCheckpoint(filepath='modelv2_disjoint_1.hdf5', verbose=1, save_best_only=False)
  csv_logger1 = CSVLogger('modelv2_disjoint_1.log')
  history = modelv2_1.fit_generator(train1_generator, 
                    steps_per_epoch = nb_train1_samples // batch_size,      
                    validation_data=validation2_generator,
                    validation_steps=nb_validation2_samples // batch_size,  
                    epochs=100,                                             
                    verbose=1,
                    callbacks=[csv_logger1, checkpointer1])
  
  modelv2_1.save('modelv2_disjoint_1.hdf5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:160: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/200
103/103 [==============================] - ETA: 0s - loss: 8.2081 - sparse_categorical_accuracy: 0.0079
Epoch 1: saving model to modelv2_disjoint_1.hdf5
103/103 [==============================] - 131s 509ms/step - loss: 8.2081 - sparse_categorical_accuracy: 0.0079 - val_loss: 7.2265 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/200
103/103 [==============================] - ETA: 0s - loss: 5.7104 - sparse_categorical_accuracy: 0.0323
Epoch 2: saving model to modelv2_disjoint_1.hdf5
103/103 [==============================] - 49s 478ms/step - loss: 5.7104 - sparse_categorical_accuracy: 0.0323 - val_loss: 6.1227 - val_sparse_categorical_accuracy: 0.0027
Epoch 3/200
103/103 [==============================] - ETA: 0s - loss: 4.5942 - sparse_categorical_accuracy: 0.0530
Epoch 3: saving model to modelv2_disjoint_1.hdf5
103/103 [==============================] - 49s 473ms/step - loss: 4.5942 - sparse_categorical_accuracy: 0.0530 - val_loss: 5.9037 - val_sparse_categorical_a

OSError: ignored

In [ ]:
moat2=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

k11 = moat2.output["stage1"]
k12 = moat2.output["res1"]
k21 = moat2.output["stage2"]
k22 = moat2.output["res2"]
k31 = moat2.output["stage3"]
k32 = moat2.output["res3"]
k41 = moat2.output["stage4"]
k42 = moat2.output["res4"]
k51 = moat2.output["stage5"]
k52 = moat2.output["res5"]

l1 = k11+k12
l2 = k21+k22
l3 = k31+k32
l4 = k41+k42
l5 = k51+k52

f1=Conv2D(32,(3,3),activation='relu')(l1)
f2=MaxPooling2D(2,2)(f1) 
f3=Conv2D(40,(3,3),activation='relu')(f2)
f4=MaxPooling2D(2,2)(f3) 
f5=Conv2D(48,(3,3),activation='relu')(f4)
f6=MaxPooling2D(2,2)(f5) 
f7=Conv2D(56,(3,3),activation='relu')(f6)
f8=MaxPooling2D(2,2)(f7)
f9=Conv2D(64,(3,3),activation='relu')(f8)
f10=MaxPooling2D(2,2)(f9)
f11=Conv2D(72,(2,2),activation='relu')(f10)
ff1=Flatten(name='flatten_layer1')(f11)

g1=Conv2D(32,(3,3),activation='relu')(l2)
g2=MaxPooling2D(2,2)(g1) 
g3=Conv2D(40,(3,3),activation='relu')(g2)
g4=MaxPooling2D(2,2)(g3) 
g5=Conv2D(48,(3,3),activation='relu')(g4)
g6=MaxPooling2D(2,2)(g5) 
g7=Conv2D(56,(3,3),activation='relu')(g6)
g8=MaxPooling2D(2,2)(g7)
g9=Conv2D(64,(2,2),activation='relu')(g8)
gg1=Flatten(name='flatten_layer2')(g9)

h1=Conv2D(64,(3,3),activation='relu')(l3)
h2=MaxPooling2D(2,2)(h1) 
h3=Conv2D(80,(3,3),activation='relu')(h2)
h4=MaxPooling2D(2,2)(h3) 
h5=Conv2D(96,(3,3),activation='relu')(h4)
h6=MaxPooling2D(2,2)(h5) 
h7=Conv2D(112,(2,2),activation='relu')(h6)
hh1=Flatten(name='flatten_layer3')(h7)

i1=Conv2D(128,(3,3),activation='relu')(l4)
i2=MaxPooling2D(2,2)(i1) 
i3=Conv2D(160,(3,3),activation='relu')(i2)
i4=MaxPooling2D(2,2)(i3) 
i5=Conv2D(200,(2,2),activation='relu')(i4)
ii1=Flatten(name='flatten_layer4')(i5)

j1=Conv2D(256,(3,3),activation='relu')(l5)
j2=MaxPooling2D(2,2)(j1) 
j3=Conv2D(320,(3,3),activation='relu')(j2)
jj1=Flatten(name='flatten_layer5')(j3)

fc2=tf.concat([ff1,gg1,hh1,ii1,jj1],axis=1)
out2=Dense(units=101, activation='softmax')(fc2)

def create_model():
  return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
  modelv2_1 = create_model()
  modelv2_1.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer1 = ModelCheckpoint(filepath='modelv2_disjoint_2.hdf5', verbose=1, save_best_only=False)
  csv_logger1 = CSVLogger('modelv2_disjoint_2.log')
  history = modelv2_1.fit_generator(train2_generator, 
                    steps_per_epoch = nb_train2_samples // batch_size,      
                    validation_data=validation2_generator,
                    validation_steps=nb_validation2_samples // batch_size,  
                    epochs=200,                                             
                    verbose=1,
                    callbacks=[csv_logger1, checkpointer1])
  
  modelv2_1.save('modelv2_disjoint_2.hdf5')

## 모델 평가 코드

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
batch_size = 32
num_of_test_samples = nb_test_samples

with tf.device('/device:GPU:0'):
  predictions = model1.predict_generator(test_generator)

y_pred = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())

print(class_labels)

print(confusion_matrix(test_generator.classes, y_pred))

report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

In [ ]:
y_pred==(true_classes+1)

## (Just Flatten)(더미코드)

In [ ]:
tf.debugging.set_log_device_placement(True)

moat2=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

from tensorflow.keras.layers import (Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten)
from tensorflow import keras

from adabelief_tf import AdaBeliefOptimizer
opt = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=1e-14, rectify=True)

o11 = moat2.output["stage1"]
o12 = moat2.output["res1"]
o21 = moat2.output["stage2"]
o22 = moat2.output["res2"]
o31 = moat2.output["stage3"]
o32 = moat2.output["res3"]
o41 = moat2.output["stage4"]
o42 = moat2.output["res4"]
o51 = moat2.output["stage5"]
o52 = moat2.output["res5"]

u1 = o11+o12
u2 = o21+o22
u3 = o31+o32
u4 = o41+o42
u5 = o51+o52

r1 = tf.keras.layers.Flatten(name='flatten_layer1')(u1)
r2 = tf.keras.layers.Flatten(name='flatten_layer2')(u2)
r3 = tf.keras.layers.Flatten(name='flatten_layer3')(u3)
r4 = tf.keras.layers.Flatten(name='flatten_layer4')(u4)
r5 = tf.keras.layers.Flatten(name='flatten_layer5')(u5)

v = tf.concat([r1,r2,r3,r4,r5],axis=1)
out2 = tf.keras.layers.Dense(
    units=101, activation='softmax', name='output_layer')(v)

def create_model():
  return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
  model2 = create_model()
  model2.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='best_model_3class_sept.hdf5', verbose=1, save_best_only=True)
  csv_logger = CSVLogger('history.log')
  history = model1.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=50,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
  
  model2.save('model2_trained.h5')




In [ ]:
from google.colab import files
files.download('model_trained.h5')